# Fourth submission: Support Vector Regression
 * min observations in each category: 2
 * 

In [8]:
%matplotlib inline
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# set seeds
np.random.seed(1)
random.seed(1)

# Clean data

## Create and drop features, manage categorical

In [9]:
# Function to prepare data

from sklearn.base import TransformerMixin

def create_new_features(X):
    # transform utc offset:
    X['Sin_UTC'] = np.sin((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    X['Cos_UTC'] = np.cos((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    
    # time since creation in days
    duration = (pd.to_datetime('today') - 
                pd.to_datetime(X['Profile Creation Timestamp']).dt.tz_localize(None))
    X['Duration'] = duration.apply(lambda x: x.days)
    
    # convert personal url into True/False (NaNs or unique)
    X['Has Personal URL'] = X['Personal URL'].notna()
    
    # log(x + 1) of numerical features
    for feature in ['Num of Followers', 'Num of People Following',
                    'Num of Status Updates', 'Num of Direct Messages',
                    'Avg Daily Profile Visit Duration in seconds',
                    'Avg Daily Profile Clicks']:
        X[f'log {feature}'] = np.log1p(X[feature])
    return X
    
def clean_features(X):
    # merge categories names with and without cap letter
    X['Location Public Visibility'] = X['Location Public Visibility'].str.lower()
    X.loc[X['Profile Category'] == " ", 'Profile Category'] = 'unknown'
    return X

def drop_features(X, l_features):
    return X.drop(columns=l_features)
    
class RemoveCategories(TransformerMixin):
    def __init__(self, min_obs=10, l_cols=[]):
        self.min_obs = min_obs
        self.l_cats = {}
        self.l_cols = l_cols

    def fit(self, X):
        # assumes all columns of df_cat are strings
        for col in self.l_cols:
            val_counts = X[col].fillna('missing').value_counts()
            self.l_cats[col] = val_counts[val_counts >= self.min_obs].index.tolist()
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        for col in self.l_cols:
            X.loc[~ X[col].isin(self.l_cats[col]), col] = 'other'        
        return X

In [37]:
# data cleaning and feature engineering

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

l_cat_small = ['Profile Text Color', 'Profile Page Color',
               'Profile Theme Color', 'User Language', 'Location']
l_drop = ['Id', 'User Name', 'Profile Image', 'Personal URL',
          'Profile Creation Timestamp', 'Location', 'UTC Offset']
l_drop_logs = ['Num of Followers', 'Num of People Following',
               'Num of Status Updates', 'Num of Direct Messages',
               'Avg Daily Profile Visit Duration in seconds', 
               'Avg Daily Profile Clicks']

features_to_drop =dict(l_features = l_drop + l_drop_logs)

pipe = Pipeline([('create new features',
                  FunctionTransformer(create_new_features)),
                 ('clean data', FunctionTransformer(clean_features)),
                 ('remove categories', RemoveCategories(min_obs=10, 
                                                        l_cols=l_cat_small)),
                 ('drop variables', FunctionTransformer(drop_features,
                                    kw_args=features_to_drop))
                ])

## Imputation and encoding/scaling

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

# categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [40]:
# train data
df = pd.read_csv('data/train.csv')
y = np.log1p(df['Num of Profile Likes'].values)
X = df.drop(columns='Num of Profile Likes')

X = pipe.fit_transform(X)

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer,
                   X.select_dtypes(exclude=['object', 'bool']).columns),
                  ('cat', categorical_transformer,
                   X.select_dtypes(include=['object', 'bool']).columns)])

X = preprocessor.fit_transform(X)

In [41]:
# to get the columns names back

from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin
def get_feature_out(estimator, feature_in):
    if hasattr(estimator,'get_feature_names'):
        if isinstance(estimator, _VectorizerMixin):
            # handling all vectorizers
            return [f'vec_{f}' for f in estimator.get_feature_names()]
        else:
            return estimator.get_feature_names(feature_in)
    elif isinstance(estimator, SelectorMixin):
        return np.array(feature_in)[estimator.get_support()]
    else:
        return feature_in
def get_ct_feature_names(ct):
    # handles all estimators, pipelines inside ColumnTransfomer
    # doesn't work when remainder =='passthrough'
    # which requires the input column names.
    output_features = []
    for name, estimator, features in ct.transformers_:
        if name!='remainder':
            if isinstance(estimator, Pipeline):
                current_features = features
                for step in estimator:
                    current_features = get_feature_out(step, current_features)
                features_out = current_features
            else:
                features_out = get_feature_out(estimator, features)
            output_features.extend(features_out)
        elif estimator=='passthrough':
            output_features.extend(ct._feature_names_in[features])
    return output_features

df_X = pd.DataFrame(X, columns=get_ct_feature_names(preprocessor))

In [35]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Columns: 224 entries, Sin_UTC to Has Personal URL_True
dtypes: float64(224)
memory usage: 12.8 MB


# Support Vector Regression

In [42]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR

svr = SVR()

parameters = {'kernel': ['rbf'],
              'C': np.linspace(0, 2, 20)}

reg_svr = RandomizedSearchCV(svr, parameters, scoring='neg_mean_squared_error',
                            n_jobs=100, random_state=None, n_iter=50, verbose=10)
reg_svr.fit(X, y)

print(f'RMSLE: {np.sqrt(-reg_svr.best_score_):.3f}',
      f'\nbest parameters: { reg_svr.best_params_}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/apps/conda/pyanni/envs/kaggle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 20 is smaller than n_iter=50. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=100)]: Done  11 out of 100 | elapsed:   16.0s remaining:  2.2min
[Parallel(n_jobs=100)]: Done  22 out of 100 | elapsed:   18.6s remaining:  1.1min
[Parallel(n_jobs=100)]: Done  33 out of 100 | elapsed:   44.7s remaining:  1.5min
[Parallel(n_jobs=100)]: Done  44 out of 100 | elapsed:   51.3s remaining:  1.1min
[Parallel(n_jobs=100)]: Done  55 out of 100 | elapsed:   52.6s remaining:   43.0s
[Parallel(n_jobs=100)]: Done  66 out of 100 | elapsed:   53.1s remaining:   27.3s
[Parallel(n_jobs=100)]: Done  77 out of 100 | elapsed:   54.3s remaining:   16.2s
[Parallel(n_jobs=100)]: D

RMSLE: 1.737 
best parameters: {'kernel': 'rbf', 'C': 1.3684210526315788}


# Submission

In [27]:
def create_submission(estimator, number):
    X_test = pd.read_csv('data/test.csv')
    Id = X_test['Id']
    X_test = pipe.transform(X_test)
    X_test = preprocessor.transform(X_test)
    y_pred = (estimator.predict(X_test))
    y_pred_test = pd.Series(np.expm1(y_pred))
    submission = pd.DataFrame({'Id': Id, 'Predicted': y_pred_test})
    path = f'submissions/submission{number}.csv'
    while os.path.exists(path):
        number += 1
        path = f'submissions/submission{number}.csv'
    submission.to_csv(path, index=False)
    print(f'submission #{number} created')

In [28]:
create_submission(reg_svr, 1)

submission #4 created


In [29]:
pd.read_csv('submissions/submission4.csv').head()

,Id,Predicted
0,49I3SOKLI2CMNGP4,2809.866389
1,727IRIR59A3P88LK,3132.848317
2,LN95SD15SRPCEE8F,327.989895
3,TB11I7F0PN033D4T,5138.042429
4,32PSGCK5PATHMR07,271.653679
